In [1]:
import pandas as pd
import pickle
import numpy as np
import pandas as pd
import nltk
import re
import codecs
from sklearn import feature_extraction
from nltk.stem.snowball import SnowballStemmer
import scipy.spatial.distance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


#read the files containing Reuters and NYT articles 
nyt_unique=pd.read_csv('nyt_uniques_topicsLabeled.csv')
reuters_unique = pickle.load( open( "20180516-20180621_reuters_unique.pkl", "rb" ) )

#this is a function to both tokenize and stem a given body of text
def token_stem(text):
    # tokenize by sentence and word. this way you ensure you get rid of punctuations
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens_with_letters = []
    # use the regex library to search only for items that contain letters. 
    #this will enable you to eliminate punctuation
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            tokens_with_letters.append(token)
    stems = [stemmer.stem(t) for t in tokens_with_letters] #"stems" part
    return stems

#reset indices for both dataframes 
#also select only article that are not labeled as "Other" from the NYT dataframe
reuters_unique.reset_index(inplace=True)
reuters_unique.drop('index',axis=1,inplace=True)
nyt_labeled_only=nyt_unique.loc[nyt_unique['Section'] != 'Other'].copy()
nyt_labeled_only.reset_index(inplace=True)
nyt_labeled_only.drop('index',axis=1,inplace=True)

#put the Reuters and NYT articles in lists
nyt_articles=nyt_labeled_only['Article'].tolist()
reuters_articles=reuters_unique['article'].tolist()

#make one big list of NYT articles and Reuters articles 
nyt_articles.extend(reuters_articles)

#These articles have weird characters in them. This is probably an encoding issue 
#Resort to cleaning them manually. 
nyt_articles=list(map(lambda unicode_line:unicode_line.translate({ord(c): None for c in 'Äòôúîù'}),nyt_articles))

#Use the nltk library's stopwords corpus and SnowballStemmer's stemmer 
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english") 

#vectorize the articles. this is where you specify important parameters such as 
#what type of tokenizer to use, whether you use idf or not etc.
vectorizer = TfidfVectorizer(stop_words='english',use_idf=True, norm='l2',tokenizer=token_stem, ngram_range=(1,3))

#put the vectors in a matrix 
tfidf_matrix = vectorizer.fit_transform(nyt_articles)

#separate the matrix into one for NYT and another for Reuter articles. 
tfidf_1=tfidf_matrix[0:1400,:]
tfidf_2=tfidf_matrix[1400:,:]

#calculate cosine similarities between Reuters and NYT articles 
similarities_dense = cosine_similarity(tfidf_1,tfidf_2,dense_output=True)

#we need the distances. so subtract similarities from 1
article_distances=1-article_similarities

#now, create a new column in your big_df and call it article_distances
big_df = pickle.load( open( "big_df.pkl", "rb" ) )
flatten = lambda l: [item for sublist in l for item in sublist]
big_df['article_distances']=pd.Series(flatten(article_distances.tolist()))

#dump big_df back to where it belongs
wf=open('big_df.pkl','wb')
pickle.dump(big_df,wf)


<class 'scipy.sparse.csr.csr_matrix'>
CPU times: user 5.18 s, sys: 416 ms, total: 5.59 s
Wall time: 2.61 s


347046